In [6]:
%load_ext autoreload
%autoreload 2

import pyscan as ps
from pyscan.drivers.testing.auto_test_driver import test_driver
from pyvisa import ResourceManager, VisaIOError
from time import sleep

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of pyscan.drivers.stanford830 failed: Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 371, in update_property
    update_generic(old.fget, new.fget)
  File "

In [7]:
rm = ResourceManager()

rs = rm.list_resources()
# print(rs)
for r in rs:
    res = rm.open_resource(r)
    try:      
        name = res.query('*IDN?')
        if 'SR830' in name:
            print(r, name)
            break
    except VisaIOError:
        pass
    res.close()

GPIB0::8::INSTR Stanford_Research_Systems,SR830,s/n86813,ver1.07 



In [8]:
name

'Stanford_Research_Systems,SR830,s/n86813,ver1.07 \n'

In [13]:
srs830 = ps.Stanford830(res)

test_driver(srs830)

[autoreload of pyscan.drivers.stanford830 failed: Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\ProgramData\anaconda3\Lib\site-packages\IPython\extensions\autoreload.py", line 371, in update_property
    update_generic(old.fget, new.fget)
  File "

Initial state for the Stanford830 was: [('phase', 180.0), ('reference_source', 'external'), ('frequency', 1000.0), ('reference_slope', 'sine zero'), ('harmonic', 1), ('instrument_amplitude', 0.984), ('input_configuration', 'A'), ('input_ground', 'AC'), ('input_coupling', 'AC'), ('input_line_filter', 'none'), ('sensitivity', 2e-09), ('reserve_mode', 'high'), ('time_constant', 30), ('filter_slope', 6), ('synchronous_filter', 'off'), ('sample_rate', 0.25), ('end_buffer_mode', 'one shot'), ('trigger_mode', 'on')]
These blacklisted settings and their corresponding values were not reset:  [('_input_configuration', 'A'), ('_time_constant', 30), ('_sample_rate', 0.25)]
Reset state for the Stanford830 was: [('phase', 180.0), ('reference_source', 'external'), ('frequency', 1000.0), ('reference_slope', 'sine zero'), ('harmonic', 1), ('instrument_amplitude', 0.984), ('input_configuration', 'A'), ('input_ground', 'AC'), ('input_coupling', 'AC'), ('input_line_filter', 'none'), ('sensitivity', 2e-09)

In [21]:
# Test get and set display

srs830.set_display(1, 'r', 'aux1')
print(srs830.get_display(1))
srs830.set_display(1, 'x', 'none')
print(srs830.get_display(1))
try:
    srs830.set_display(1, 'y', 'none')
except AssertionError:
    print('Sucessfully failed setting channel 1 to y')
srs830.set_display(2, 'theta', 'aux3')
print(srs830.get_display(2))
srs830.set_display(2, 'y', 'none')
print(srs830.get_display(2))
try:
    srs830.set_display(2, 'x', 'none')
except AssertionError:
    print('Sucessfully failed setting channel 2 to x')

('r', 'aux1')
('x', 'none')
Sucessfully failed setting channel 1 to y
('theta', 'aux3')
('y', 'none')
Sucessfully failed setting channel 2 to x


In [23]:
# Test get and set offset and expand

srs830.set_channel_offset_expand(1, 100, 10)
print(srs830.get_channel_offset_expand(1))
srs830.set_channel_offset_expand(1, 0, 1)
print(srs830.get_channel_offset_expand(1))


(100.0, 1)
(0.0, 0)


In [68]:
# Test auto offset
# First number should be some value, second number should be close to 0
srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('x')
srs830.auto_offset('x')
v_offset = srs830.read('x')
print(v0, v_offset)
srs830.set_channel_offset_expand(1, 0, 1)


srs830.set_channel_offset_expand(2, 0, 1)
v0 = srs830.read('y')
srs830.auto_offset('y')
v_offset = srs830.read('y')
print(v0, v_offset)
srs830.set_channel_offset_expand(2, 0, 1)

srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('r')
srs830.auto_offset('r')
v_offset = srs830.read('r')
print(v0, v_offset)
srs830.set_display(1, 'r', 'none')
srs830.set_channel_offset_expand(1, 0, 1)
srs830.set_display(1, 'x', 'none')


0.00384523 6.10356e-05
-0.0144044 -0.000183106
0.0 0.0


In [33]:
# Test read aux inputs
# shoudl show some random small value
for i in range(1, 5):
    print(srs830.read_aux_input(i))

0.00366667
0.00266667
0.00433333
0.00266667


In [36]:
# Read x, y, r, theta

print(srs830.read('x'))
print(srs830.read('y'))
print(srs830.read('r'))
print(srs830.read('theta'))

0.00384523
-0.0144044
0.0
-74.97


In [42]:
# Read display

for i in range(1, 3):
    print(srs830.read_display(i))

0.00384523
-0.0144044


In [64]:
# Test snap

for source in ['y', 'r', 'theta', 'aux1', 'aux2', 'aux3',
                  'aux4', 'frequency', 'display1', 'display2']:
    print(srs830.snap('x', source))

srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1')

try:
    srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1', 'display2')
except:
    print('Couldnt snap 7 entries')

[0.00384523, -0.0144044]
[0.00384523, 0.0]
[0.00384523, -74.97]
[0.00384523, 0.000666667]
[0.00384523, 0.004]
[0.00384523, -0.00633333]
[0.00384523, 0.000666667]
[0.00384523, 1000.0]
[0.00384523, 0.00384523]
[0.00384523, -0.0144044]
Couldnt snap 7 entries


In [67]:
# test get ID

srs830.get_identificaiton_string()

'Stanford_Research_Systems,SR830,s/n86813,ver1.07 '

In [95]:
# test buffer_points

srs830.buffer_points
srs830.wait_for_trigger()
srs830.set_buffer_mode(0.25)
srs830.wait_for_trigger()
# srs830.start()
srs830.trigger()
sleep(20)
srs830.buffer_points


5

In [91]:
srs830.query('SRAT?')

'0\n'

In [92]:
srs830.sample_rate

0.0625